# Use Pandas' get_chunk function to handle millions of data


- Download ows-raw.txt
- Read data as chunk

In [1]:
file_path = 'E:/2024计算社会科学夏校/data/ows-raw.txt'

In [2]:
import pandas as pd

In [3]:
f = open(file_path, 'r', encoding='utf-8')
reader = pd.read_table(f,  sep=',',  quotechar='"', iterator=True, on_bad_lines='skip') #跳过报错行
chunkSize = 10**4
chunk = reader.get_chunk(chunkSize)
len(chunk)

10000

In [4]:
chunkSize = 10**4
total_rows = 0

chunk_reader = pd.read_csv(file_path, sep=',', quotechar='"', 
                           iterator=True, on_bad_lines='skip', chunksize=chunkSize)
for chunk in chunk_reader:
    total_rows += len(chunk)
    
print(f'Total number of rows: {total_rows}')

Total number of rows: 6602120


In [5]:
chunk.head()

,Twitter ID,Text,Profile Image URL,Day,Hour,Minute,Created At,Geo,From User,From User ID,Language,To User,To User ID,Source
6600000,170726983490211841,Stand Up Mr. US Business Man and take responsi...,http://a2.twimg.com/profile_images/1752607483/...,2012-02-18,4,30,2012-02-18 04:30:59,N;,bentley_cat,463108759,en,NaN,0,&lt;a href=&quot;http://www.bestoftheinternets...
6600001,170727024841854976,RT @C0d3Fr0sty: MT( Link shortened) @Kaymee: I...,http://a2.twimg.com/profile_images/1599465487/...,2012-02-18,4,31,2012-02-18 04:31:09,N;,marylouise996S,15380166,en,NaN,0,&lt;a href=&quot;http://www.tweetdeck.com&quot...
6600002,170727037370253312,China had an #ows before everyone else 1989 Ti...,http://a0.twimg.com/profile_images/1302276340/...,2012-02-18,4,31,2012-02-18 04:31:12,N;,dfwlibrarian,17644162,en,NaN,0,&lt;a href=&quot;http://janetter.net/&quot; re...
6600003,170727054361362433,"Currency, Capital and Evolution: - http://t.co...",http://a3.twimg.com/profile_images/1597982571/...,2012-02-18,4,31,2012-02-18 04:31:16,N;,OmniusManifesto,394061184,it,NaN,0,&lt;a href=&quot;http://www.socialoomph.com&qu...
6600004,170727082391900160,Our problems rise much more from Govts corrupt...,http://a0.twimg.com/profile_images/1592676372/...,2012-02-18,4,31,2012-02-18 04:31:23,N;,IndyPolitico,73935439,en,NaN,0,&lt;a href=&quot;http://www.socialoomph.com&qu...


- Print out the number of tweets and unique users for each day

In [6]:
chunkSize = 10**4

# Create a dataframe reader object
chunk_reader = pd.read_csv(file_path, sep=',', quotechar='"', 
                           iterator=True, on_bad_lines='skip', chunksize=chunkSize)

# Initialize an empty DataFrame to store the group counts
group_counts = pd.DataFrame()

# Column name to group by
group_column = 'Day'  # Replace with the actual column name
id_column = 'From User ID'  # Replace with the actual column name for IDs

# Iterate over chunks
for chunk in chunk_reader:
    # Group by the specified column and count the observations in each group
    chunk_group_counts1 = chunk.groupby(group_column).size().reset_index(name='count')
    
    # Group by the specified column and calculate the number of unique IDs in each group
    chunk_group_counts2 = chunk.groupby(group_column)[id_column].nunique().reset_index(name='unique_id_count')
    
    # Append the group counts to the total group counts
    group_counts = pd.concat([group_counts, chunk_group_counts1, chunk_group_counts2], ignore_index=True)

# After processing all chunks, group again to consolidate counts across chunks
final_group_counts = group_counts.groupby(group_column).sum().reset_index()

# Sort the final group counts
final_group_counts = final_group_counts.sort_values(by='Day', ascending=True)

print(final_group_counts)

            Day    count  unique_id_count
0    2011-10-06  49638.0          24832.0
1    2011-10-07  65238.0          33883.0
2    2011-10-08  65949.0          33204.0
3    2011-10-09  65097.0          32201.0
4    2011-10-10  78619.0          41465.0
..          ...      ...              ...
131  2012-02-14  13580.0           5349.0
132  2012-02-15  13248.0           5598.0
133  2012-02-16  12837.0           5083.0
134  2012-02-17  12468.0           4897.0
135  2012-02-18   4859.0           2273.0

[136 rows x 3 columns]
